Run the Dickey-Fuller test on a sample of 10% of US counties in the year 2022.

In [1]:
%%capture
!pip install statsmodels vaex

In [16]:
from pathlib import Path
import math

import numpy as np
import pandas as pd
import vaex as vx
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller

# HOME = Path(os.environ['HOME'])
HOME = Path("/notebooks")
# HOME = Path("/mnt/c/Users/isabe")
# PROJECT = HOME / "Documents/repos/eagle-comp"
PROJECT = HOME / "eagle-comp"
DATA = PROJECT / "data"
EAGLE_DATA = Path("/datasets/eagle-comp")

In [3]:
def add_zeroes(outage_df):
    outage_df = outage_df.reset_index()
    outage_df['run_start_time'] = pd.to_datetime(outage_df['run_start_time'])
    outage_df['run_start_time'] = outage_df['run_start_time'].dt.round('15min')
    outage_df = outage_df.set_index(["fips_code", "run_start_time"]).sort_index()
    na_rows = outage_df[outage_df['sum'].isna()].index
    outage_df = outage_df.groupby('fips_code').apply(lambda g: g.loc[g.index.get_level_values(0)[0]].resample('15T').first().fillna(0))
    outage_df.loc[na_rows] = np.nan
    outage_df.reset_index(inplace=True)
    return outage_df

In [20]:
df_2022 = pd.read_csv(EAGLE_DATA / "eaglei_outages_2022.csv", parse_dates=["run_start_time"], index_col=["fips_code", "run_start_time"]).sort_index()

In [21]:
# Since there are 3044 FIPS codes, we can sample 10% of them to get a sense of the data
all_fips = df_2022.index.get_level_values(0).unique()
sample_size = math.ceil(all_fips.size * 0.1) # get 10% of values
# set seed
np.random.seed(123)
fips_sample = np.random.choice(all_fips, size=sample_size, replace=False)
fips_sample[:10]

array([53013, 25017, 26085, 55057,  8045, 46017, 12101, 20101, 53019,
       48299])

In [22]:
sample = df_2022.loc[fips_sample]
sample.sort_index(inplace=True)

In [8]:
def dickey_fuller(series):
    dftest = adfuller(series.dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', 'Number of Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    return dfoutput

In [42]:
def dickey_fuller_df(series, fips):
    results = pd.DataFrame(index=fips_sample, columns=['Test Statistic', 'p-value', '# Lags Used', '# Observations Used', 'Critical Value (1%)', 'Critical Value (5%)', 'Critical Value (10%)'])
    for fips in tqdm(fips_sample):
        if series.loc[fips].dropna().size > 0:
            results.loc[fips] = dickey_fuller(series.loc[fips])
        else:
            results.loc[fips] = np.nan
    return results

In [43]:
dickey_fuller_results = dickey_fuller_df(sample['sum'], fips_sample)

 96%|█████████▌| 292/305 [01:23<00:04,  3.21it/s]/opt/conda/envs/rapids/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:924: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
100%|██████████| 305/305 [01:27<00:00,  3.48it/s]


In [ ]:
dickey_fuller_results.to_csv('dickey_fuller_results.csv')

In [44]:
(dickey_fuller_results['p-value'] < 0.01).sum()

285

In [63]:
(dickey_fuller_results['p-value'] < 0.1).sum()

292

In [61]:
dickey_fuller_results.shape

(305, 7)

In [54]:
non_stationary = dickey_fuller_results['p-value'] > 0.01
non_stationary_fips = dickey_fuller_results[non_stationary].index.values

In [98]:
# use groupby to return the number of nan values and number of rows for each fips code
num_observations = (~(df_2022.loc[non_stationary_fips][['sum']].isna())).groupby("fips_code").apply(lambda x: pd.DataFrame({"Number of Observations": x.sum()})).reset_index().drop(columns="level_1").set_index("fips_code")

In [128]:
res = num_observations.merge(dickey_fuller_results['p-value'], left_index=True, right_index=True, how='inner').sort_values('p-value', ascending=False)

In [129]:
res.index.name = 'County FIPS Code'

In [130]:
res['p-value'] = res['p-value'].astype(float)

In [132]:
res['p-value'] = res['p-value'].round(3)

In [134]:
res.T

County FIPS Code,38011,31023,1071,8111,15009,46071,38081,31049,46065,46061,72113,27121,31003
Number of Observations,72.000,305.000,122.00,91.000,282.000,1150.000,4612.000,176.000,175.000,1760.000,28390.000,1533.000,177.000
p-value,0.746,0.574,0.49,0.252,0.165,0.109,0.096,0.083,0.055,0.033,0.021,0.019,0.011
